In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from scipy.stats import linregress

from config import api_key


In [33]:
# Load base air quality and health data
AQ_health_data = "resources/AQ_and_health.csv"

AQ_health_df = pd.read_csv(AQ_health_data)

AQ_health_df.head()

,State,City,FIPS,Pop 2010,Cancer %,Asthma %,Heart Disease %,Pulminary Disease %,Mental Illness %,Poor Health %,...,Pop_Metro_2015,CO2 1975,CO2 1990,CO2 2000,CO2 2012,PM2.5 2000,PM2.5 2005,PM 2.5 2010,PM2.5 2014,Heat Wave Index
0,NM,Albuquerque,3502000,545852,5.8,10.4,5.6,5.5,12.6,12.9,...,6.859337e+05,4.360884e+06,3.720008e+06,3.800904e+06,3.259878e+06,8.652916,8.211202,6.974018,6.863455,10.98080
1,AK,Anchorage,203000,291826,5.1,9.3,4.5,5.0,11.4,10.6,...,1.230898e+05,3.779778e+05,3.344264e+05,3.823933e+05,3.491618e+05,5.862218,5.384438,4.634226,8.126340,9.12966
2,GA,Atlanta,1304000,420003,4.8,9.0,5.0,5.7,12.2,11.1,...,1.516952e+06,1.272662e+07,1.229068e+07,1.459993e+07,1.250880e+07,19.794460,13.311600,10.708480,10.280250,18.74350
3,TX,Austin,4805000,790390,4.3,8.3,3.9,4.0,11.4,9.4,...,1.063621e+06,4.417963e+06,4.107195e+06,5.003950e+06,4.174131e+06,15.143920,10.114128,8.038374,8.593885,15.35470
4,CA,Bakersfield,603526,347483,5.0,9.5,5.4,6.1,14.0,13.1,...,5.333185e+05,3.324191e+06,3.342471e+06,3.438174e+06,2.668842e+06,16.668700,14.384460,12.763120,15.111900,4.47877


In [24]:
# Load econ data for merging

econ_data = "resources/econ_data.csv"

econ_df = pd.read_csv(econ_data)

econ_df.head()

,State,City,Metro,FIPS,Pop 2010,Poverty Rate 2009-2013,Economic Rank
0,TX,Austin,Austin,4805000,790390,0.191,3
1,CA,San Francisco,San Francisco,667000,805235,0.135,4
2,TX,Dallas,Dallas,4819000,1197816,0.238,5
3,NC,Raleigh,Raleigh,3755000,403892,0.162,6
4,FL,Orlando,Orlando,1253000,238300,0.192,7


In [34]:
# Merge econ and health data on FIPS code

AQ_health_econ_df = pd.merge(AQ_health_df,econ_df[['FIPS','Poverty Rate 2009-2013','Economic Rank']],on='FIPS', how='inner')

AQ_health_econ_df.head()

,State,City,FIPS,Pop 2010,Cancer %,Asthma %,Heart Disease %,Pulminary Disease %,Mental Illness %,Poor Health %,...,CO2 1990,CO2 2000,CO2 2012,PM2.5 2000,PM2.5 2005,PM 2.5 2010,PM2.5 2014,Heat Wave Index,Poverty Rate 2009-2013,Economic Rank
0,NM,Albuquerque,3502000,545852,5.8,10.4,5.6,5.5,12.6,12.9,...,3.720008e+06,3.800904e+06,3.259878e+06,8.652916,8.211202,6.974018,6.863455,10.98080,0.179,125
1,AK,Anchorage,203000,291826,5.1,9.3,4.5,5.0,11.4,10.6,...,3.344264e+05,3.823933e+05,3.491618e+05,5.862218,5.384438,4.634226,8.126340,9.12966,0.079,197
2,GA,Atlanta,1304000,420003,4.8,9.0,5.0,5.7,12.2,11.1,...,1.229068e+07,1.459993e+07,1.250880e+07,19.794460,13.311600,10.708480,10.280250,18.74350,0.250,17
3,TX,Austin,4805000,790390,4.3,8.3,3.9,4.0,11.4,9.4,...,4.107195e+06,5.003950e+06,4.174131e+06,15.143920,10.114128,8.038374,8.593885,15.35470,0.191,3
4,CA,Bakersfield,603526,347483,5.0,9.5,5.4,6.1,14.0,13.1,...,3.342471e+06,3.438174e+06,2.668842e+06,16.668700,14.384460,12.763120,15.111900,4.47877,0.204,162


In [35]:
#This has the census number codes for states, needed to access US Census API
state_FIPS = "resources/state-fips.csv"

state_FIPS_df = pd.read_csv(state_FIPS)

state_FIPS_df.head()

,stname,state_fips,State
0,Alabama,1,AL
1,Alaska,2,AK
2,Arizona,4,AZ
3,Arkansas,5,AR
4,California,6,CA


In [36]:
# Takes last five digits of FIPS and creates"FIPS_place", to match census FIPS for places
AQ_health_econ_df['FIPS'] = AQ_health_econ_df['FIPS'].astype(str)

AQ_health_econ_df['FIPS_place'] = AQ_health_econ_df['FIPS'].str[-5:]

In [37]:
AQ_health_econ_df.head()

,State,City,FIPS,Pop 2010,Cancer %,Asthma %,Heart Disease %,Pulminary Disease %,Mental Illness %,Poor Health %,...,CO2 2000,CO2 2012,PM2.5 2000,PM2.5 2005,PM 2.5 2010,PM2.5 2014,Heat Wave Index,Poverty Rate 2009-2013,Economic Rank,FIPS_place
0,NM,Albuquerque,3502000,545852,5.8,10.4,5.6,5.5,12.6,12.9,...,3.800904e+06,3.259878e+06,8.652916,8.211202,6.974018,6.863455,10.98080,0.179,125,02000
1,AK,Anchorage,203000,291826,5.1,9.3,4.5,5.0,11.4,10.6,...,3.823933e+05,3.491618e+05,5.862218,5.384438,4.634226,8.126340,9.12966,0.079,197,03000
2,GA,Atlanta,1304000,420003,4.8,9.0,5.0,5.7,12.2,11.1,...,1.459993e+07,1.250880e+07,19.794460,13.311600,10.708480,10.280250,18.74350,0.250,17,04000
3,TX,Austin,4805000,790390,4.3,8.3,3.9,4.0,11.4,9.4,...,5.003950e+06,4.174131e+06,15.143920,10.114128,8.038374,8.593885,15.35470,0.191,3,05000
4,CA,Bakersfield,603526,347483,5.0,9.5,5.4,6.1,14.0,13.1,...,3.438174e+06,2.668842e+06,16.668700,14.384460,12.763120,15.111900,4.47877,0.204,162,03526


In [38]:
# This cell merges the Census state codes into the larger dataframe

# Ensures that State keys are strings and have no empty characters
AQ_health_econ_df['State'] = AQ_health_econ_df['State'].astype(str)

state_FIPS_df['State'] = state_FIPS_df['State'].astype(str)

AQ_health_econ_df['State'] = AQ_health_econ_df['State'].str.strip()

state_FIPS_df['State'] = state_FIPS_df['State'].str.strip()

# Merges the dfs on State
merged_df = pd.merge(AQ_health_econ_df,state_FIPS_df,on='State', how='inner')

merged_df.to_csv("output_data/merged_df.csv")

merged_df.head(10)

,State,City,FIPS,Pop 2010,Cancer %,Asthma %,Heart Disease %,Pulminary Disease %,Mental Illness %,Poor Health %,...,PM2.5 2000,PM2.5 2005,PM 2.5 2010,PM2.5 2014,Heat Wave Index,Poverty Rate 2009-2013,Economic Rank,FIPS_place,stname,state_fips
0,NM,Albuquerque,3502000,545852,5.8,10.4,5.6,5.5,12.6,12.9,...,8.652916,8.211202,6.974018,6.863455,10.980800,0.179,125,02000,New Mexico,35
1,AK,Anchorage,203000,291826,5.1,9.3,4.5,5.0,11.4,10.6,...,5.862218,5.384438,4.634226,8.126340,9.129660,0.079,197,03000,Alaska,2
2,GA,Atlanta,1304000,420003,4.8,9.0,5.0,5.7,12.2,11.1,...,19.794460,13.311600,10.708480,10.280250,18.743500,0.250,17,04000,Georgia,13
3,TX,Austin,4805000,790390,4.3,8.3,3.9,4.0,11.4,9.4,...,15.143920,10.114128,8.038374,8.593885,15.354700,0.191,3,05000,Texas,48
4,TX,Corpus Christi,4817000,305215,5.4,8.6,6.5,6.0,12.2,14.1,...,13.320098,8.644316,6.791826,7.352780,14.974500,0.182,188,17000,Texas,48
5,TX,Dallas,4819000,1197816,4.8,9.4,5.6,5.8,13.2,12.8,...,15.816820,11.450900,9.357422,10.363200,11.645800,0.238,5,19000,Texas,48
6,TX,Houston,4835000,2099451,4.8,8.8,5.6,5.6,13.0,12.9,...,17.062260,11.719760,8.876372,9.491250,51.104099,0.229,119,35000,Texas,48
7,TX,Lubbock,4845000,229573,5.2,9.2,5.5,5.7,13.1,11.8,...,7.981408,6.478616,5.471234,5.813725,9.009750,0.218,86,45000,Texas,48
8,TX,San Antonio,4865000,1327407,4.9,8.4,5.9,5.4,12.5,14.1,...,14.860020,9.888844,7.945454,8.466455,10.723400,0.199,32,65000,Texas,48
9,CA,Bakersfield,603526,347483,5.0,9.5,5.4,6.1,14.0,13.1,...,16.668700,14.384460,12.763120,15.111900,4.478770,0.204,162,03526,California,6


In [39]:
# Converts FIPS codes into strings so that they can be used in API query url

merged_df['FIPS_place'] = merged_df['FIPS_place'].astype(str)

merged_df['state_fips'] = merged_df['state_fips'].astype(str)

# Sets base url for census API

url = "https://api.census.gov/data/2018/acs/acs1/profile?get=NAME,DP03_0119PE&DP03_0062E&for=place:"

# Creates a blank dataframe to store the API responses

census_df = pd.DataFrame(columns = ['City','census_place', 'FIPS_place', 'Poverty_2017','Med_Income_2017'])

# Interates through merged dataframe and queries the API for each city,  pulling poverty% and median income

for index, row in merged_df.iterrows():
    
    place_fips = row['FIPS_place']
    
    # This ensures that state FIPS has two digits, as required by API
    state_fips = row['state_fips'].zfill(2)
    
    query_url = url + place_fips + "&in=state:" + state_fips + "&key=" + api_key
    
    # query_url = "https://api.census.gov/data/2018/acs/acs1/profile?get=NAME,DP03_0119PE&DP03_0062E&for=place:19000&in=state:48&key=c6dfdc55cc6667ba133bfb11a069ed77879b6c16"
    
    census_response = requests.get(query_url)
    
    response_json = census_response.json()
    
    print(response_json)
    
    # Create a dataframe for the individual response and then append it to census_df
    response_df= pd.DataFrame(({'City' : row['City'],
                                'census_place' : response_json[1][0], 
                                'FIPS_place' : response_json[1][4],
                                'Poverty_2017' : response_json[1][1],
                                'Med_Income_2017': response_json[1][2]}), index = [0])

    census_df = census_df.append(response_df, ignore_index = True)     

census_df.to_csv("output_data/census_df.csv")   

[['NAME', 'DP03_0119PE', 'DP03_0062E', 'state', 'place'], ['Albuquerque city, New Mexico', '12.1', '51099', '35', '02000']]
[['NAME', 'DP03_0119PE', 'DP03_0062E', 'state', 'place'], ['Anchorage municipality, Alaska', '6.4', '83648', '02', '03000']]
[['NAME', 'DP03_0119PE', 'DP03_0062E', 'state', 'place'], ['Atlanta city, Georgia', '14.9', '65345', '13', '04000']]
[['NAME', 'DP03_0119PE', 'DP03_0062E', 'state', 'place'], ['Austin city, Texas', '9.4', '71543', '48', '05000']]
[['NAME', 'DP03_0119PE', 'DP03_0062E', 'state', 'place'], ['Corpus Christi city, Texas', '13.7', '56602', '48', '17000']]
[['NAME', 'DP03_0119PE', 'DP03_0062E', 'state', 'place'], ['Dallas city, Texas', '13.8', '52210', '48', '19000']]
[['NAME', 'DP03_0119PE', 'DP03_0062E', 'state', 'place'], ['Houston city, Texas', '16.9', '51203', '48', '35000']]
[['NAME', 'DP03_0119PE', 'DP03_0062E', 'state', 'place'], ['Lubbock city, Texas', '11.2', '48042', '48', '45000']]
[['NAME', 'DP03_0119PE', 'DP03_0062E', 'state', 'place'

[['NAME', 'DP03_0119PE', 'DP03_0062E', 'state', 'place'], ['Washington city, District of Columbia', '11.3', '85203', '11', '50000']]
[['NAME', 'DP03_0119PE', 'DP03_0062E', 'state', 'place'], ['Wichita city, Kansas', '10.7', '51051', '20', '79000']]


In [40]:
census_df.head()

,City,census_place,FIPS_place,Poverty_2017,Med_Income_2017
0,Albuquerque,"Albuquerque city, New Mexico",02000,12.1,51099
1,Anchorage,"Anchorage municipality, Alaska",03000,6.4,83648
2,Atlanta,"Atlanta city, Georgia",04000,14.9,65345
3,Austin,"Austin city, Texas",05000,9.4,71543
4,Corpus Christi,"Corpus Christi city, Texas",17000,13.7,56602


In [41]:
# Merges census data with other combined data to create a final df

final_df = pd.merge(merged_df,census_df,on='City', how='inner')

final_df.to_csv("output_data/final_df.csv")

final_df.head()


,State,City,FIPS,Pop 2010,Cancer %,Asthma %,Heart Disease %,Pulminary Disease %,Mental Illness %,Poor Health %,...,Heat Wave Index,Poverty Rate 2009-2013,Economic Rank,FIPS_place_x,stname,state_fips,census_place,FIPS_place_y,Poverty_2017,Med_Income_2017
0,NM,Albuquerque,3502000,545852,5.8,10.4,5.6,5.5,12.6,12.9,...,10.98080,0.179,125,02000,New Mexico,35,"Albuquerque city, New Mexico",02000,12.1,51099
1,AK,Anchorage,203000,291826,5.1,9.3,4.5,5.0,11.4,10.6,...,9.12966,0.079,197,03000,Alaska,2,"Anchorage municipality, Alaska",03000,6.4,83648
2,GA,Atlanta,1304000,420003,4.8,9.0,5.0,5.7,12.2,11.1,...,18.74350,0.250,17,04000,Georgia,13,"Atlanta city, Georgia",04000,14.9,65345
3,TX,Austin,4805000,790390,4.3,8.3,3.9,4.0,11.4,9.4,...,15.35470,0.191,3,05000,Texas,48,"Austin city, Texas",05000,9.4,71543
4,TX,Corpus Christi,4817000,305215,5.4,8.6,6.5,6.0,12.2,14.1,...,14.97450,0.182,188,17000,Texas,48,"Corpus Christi city, Texas",17000,13.7,56602
